In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

#中文字体
import matplotlib  
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


# WiFi Dictionary

In [2]:
user = pd.read_csv('./input/训练数据-ccf_first_round_user_shop_behavior.csv')
shop = pd.read_csv('./input/训练数据-ccf_first_round_shop_info.csv')
test = pd.read_csv('./input/AB榜测试集-evaluation_public.csv')

user.shape
user.head()
shop.shape
shop.head()
test.shape
test.head()

(1138015, 6)

,user_id,shop_id,time_stamp,longitude,latitude,wifi_infos
0,u_376,s_2871718,2017-08-06 21:20,122.308291,32.088040,b_6396480|-67|false;b_41124514|-86|false;b_287...
1,u_376,s_2871718,2017-08-06 21:20,122.308162,32.087970,b_6396480|-67|false;b_56328155|-73|false;b_411...
2,u_1041,s_181637,2017-08-02 13:10,117.365255,40.638214,b_8006367|-78|false;b_2485110|-52|false;b_3005...
3,u_1158,s_609470,2017-08-13 12:30,121.134451,31.197416,b_26250579|-73|false;b_26250580|-64|false;b_26...
4,u_1654,s_3816766,2017-08-25 19:50,122.255867,31.351320,b_39004150|-66|false;b_39004148|-58|false;b_21...


(8477, 6)

,shop_id,category_id,longitude,latitude,price,mall_id
0,s_26,c_4,122.346736,31.833507,57,m_690
1,s_133,c_6,121.134362,31.197511,58,m_6587
2,s_251,c_38,121.000505,30.907667,34,m_5892
3,s_372,c_30,119.864982,26.659876,44,m_625
4,s_456,c_26,122.594243,31.581499,44,m_3839


(483931, 7)

,row_id,user_id,mall_id,time_stamp,longitude,latitude,wifi_infos
0,118742,u_30097142,m_3916,2017-09-05 13:00,122.141011,39.818847,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,2017-09-06 13:10,118.191907,32.855858,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,2017-09-06 17:40,119.192110,32.424667,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,2017-09-03 12:10,120.612201,34.055249,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,2017-09-02 20:40,116.861989,40.326858,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [3]:
user = user.merge(shop, left_on='shop_id', right_on='shop_id', how='left')
user['row_id'] = -1

del shop

user = user[['row_id', 'user_id', 'mall_id', 'wifi_infos', 'shop_id']]
test = test[['row_id', 'user_id', 'mall_id', 'wifi_infos']]

user.shape
user.head()
test.shape
test.head()

(1138015, 5)

,row_id,user_id,mall_id,wifi_infos,shop_id
0,-1,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,-1,u_376,m_1409,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,-1,u_1041,m_4079,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637
3,-1,u_1158,m_6587,b_26250579|-73|false;b_26250580|-64|false;b_26...,s_609470
4,-1,u_1654,m_3005,b_39004150|-66|false;b_39004148|-58|false;b_21...,s_3816766


(483931, 4)

,row_id,user_id,mall_id,wifi_infos
0,118742,u_30097142,m_3916,b_34366982|-82|false;b_37756289|-53|false;b_41...
1,118743,u_30097803,m_5085,b_36722251|-81|false;b_10537579|-75|false;b_43...
2,118744,u_30097889,m_4033,b_30026291|-74|false;b_30026290|-74|false;b_36...
3,118745,u_30098996,m_4515,b_33412374|-77|false;b_22084893|-86|false;b_52...
4,118746,u_30099170,m_7168,b_19882704|-77|false;b_2241462|-49|false;b_585...


In [4]:
test['shop_id'] = -1

full_data = pd.concat([user, test])

full_data.shape
full_data.head()

(1621946, 5)

,row_id,user_id,mall_id,wifi_infos,shop_id
0,-1,u_376,m_1409,b_6396480|-67|false;b_41124514|-86|false;b_287...,s_2871718
1,-1,u_376,m_1409,b_6396480|-67|false;b_56328155|-73|false;b_411...,s_2871718
2,-1,u_1041,m_4079,b_8006367|-78|false;b_2485110|-52|false;b_3005...,s_181637
3,-1,u_1158,m_6587,b_26250579|-73|false;b_26250580|-64|false;b_26...,s_609470
4,-1,u_1654,m_3005,b_39004150|-66|false;b_39004148|-58|false;b_21...,s_3816766


In [5]:
del user
del test

In [6]:
full_data.wifi_infos = full_data.wifi_infos.apply(lambda x : x.split(';'))
full_data['wifi_num'] = full_data.wifi_infos.apply(len)

full_data.head()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
0,-1,u_376,m_1409,"[b_6396480|-67|false, b_41124514|-86|false, b_...",s_2871718,10
1,-1,u_376,m_1409,"[b_6396480|-67|false, b_56328155|-73|false, b_...",s_2871718,10
2,-1,u_1041,m_4079,"[b_8006367|-78|false, b_2485110|-52|false, b_3...",s_181637,10
3,-1,u_1158,m_6587,"[b_26250579|-73|false, b_26250580|-64|false, b...",s_609470,10
4,-1,u_1654,m_3005,"[b_39004150|-66|false, b_39004148|-58|false, b...",s_3816766,10


In [7]:
full_data.reset_index(drop=True, inplace=True)

full_data.tail()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
1621941,483927,u_63919729,m_7523,"[b_30877374|-71|false, b_8210864|-61|false, b_...",-1,10
1621942,483928,u_63920157,m_4828,"[b_8685813|-79|false, b_8685969|-74|false, b_1...",-1,10
1621943,483929,u_63920157,m_4828,"[b_49160464|-65|false, b_34498163|-82|false, b...",-1,8
1621944,483930,u_63920157,m_4828,"[b_45089046|-47|true, b_49160464|-65|false, b_...",-1,3
1621945,483931,u_63920806,m_7973,"[b_25163301|-69|false, b_39872252|-57|false, b...",-1,10


In [8]:
len(full_data.mall_id.value_counts())
full_data.wifi_num.value_counts()

97

10    1492325
9       30890
8       25438
7       21666
6       17442
5       13367
4        9624
3        6368
2        3303
1        1511
20         10
18          1
16          1
Name: wifi_num, dtype: int64

In [9]:
def wifi_list(x):
    a = []
    while len(x) != 0:
        b = x.pop(0)
        a.append(b.split('|'))
    return a

In [25]:
full_data.wifi_infos = full_data.wifi_infos.apply(wifi_list)

full_data.head()

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
0,-1,u_376,m_1409,"[[b_41124514, -86, false], [b_28723327, -90, f...",s_2871718,10
1,-1,u_376,m_1409,"[[b_6396480, -67, false], [b_56328155, -73, fa...",s_2871718,10
2,-1,u_1041,m_4079,"[[b_8006367, -78, false], [b_2485110, -52, fal...",s_181637,10
3,-1,u_1158,m_6587,"[[b_26250579, -73, false], [b_26250580, -64, f...",s_609470,10
4,-1,u_1654,m_3005,"[[b_39004150, -66, false], [b_39004148, -58, f...",s_3816766,10


In [ ]:
import tq

In [11]:
def shop_wifi_dict(x):
    shop_wifi = {}
    temp = full_data[full_data.shop_id == x].index
    for a in temp:
        for i in range(len(full_data.iloc[a].wifi_infos)):
            if shop_wifi.has_key(full_data.iloc[a].wifi_infos[i][0]) == True:
                shop_wifi[full_data.iloc[a].wifi_infos[i][0]] = shop_wifi[full_data.iloc[a].wifi_infos[i][0]] + int(full_data.iloc[a].wifi_infos[i][1])
            else:
                shop_wifi[full_data.iloc[a].wifi_infos[i][0]] = int(full_data.iloc[a].wifi_infos[i][1])
    return shop_wifi

def mall_wifi_dict(x):
    mall_wifi = {}
    temp = full_data[full_data.mall_id == x].index
    for a in temp:
        for i in range(len(full_data.iloc[a].wifi_infos)):
            if mall_wifi.has_key(full_data.iloc[a].wifi_infos[i][0]) == True:
                mall_wifi[full_data.iloc[a].wifi_infos[i][0]] = mall_wifi[full_data.iloc[a].wifi_infos[i][0]] + int(full_data.iloc[a].wifi_infos[i][1])
            else:
                mall_wifi[full_data.iloc[a].wifi_infos[i][0]] = int(full_data.iloc[a].wifi_infos[i][1])
    return mall_wifi

In [12]:
len(shop_wifi_dict('s_2871718'))
len(mall_wifi_dict('m_1409'))

69

7092

# Sparse Matrix

In [86]:
from tqdm import tqdm

In [43]:
temp_shop_dict = shop_wifi_dict('s_2871718')
temp_shop_dict

temp_mall_dict = mall_wifi_dict('m_1409')
temp_mall_dict

{'b_1071564': -92,
 'b_12853924': -602,
 'b_13063943': -161,
 'b_15781664': -77,
 'b_19586106': -81,
 'b_25137939': -435,
 'b_25137940': -354,
 'b_25137941': -180,
 'b_25137942': -361,
 'b_26748506': -453,
 'b_27559559': -56,
 'b_27652652': -131,
 'b_28660931': -75,
 'b_28723327': -2760,
 'b_30230709': -85,
 'b_30230710': -174,
 'b_30230712': -267,
 'b_30230713': -85,
 'b_30230714': -352,
 'b_31100463': -350,
 'b_31100514': -1243,
 'b_3145641': -88,
 'b_32053319': -3966,
 'b_32093290': -180,
 'b_33391980': -73,
 'b_39359823': -552,
 'b_39487407': -77,
 'b_41124514': -2634,
 'b_42964768': -93,
 'b_43505692': -84,
 'b_45933826': -89,
 'b_4640391': -93,
 'b_4640392': -92,
 'b_50066470': -87,
 'b_50744804': -99,
 'b_54320006': -75,
 'b_54944292': -1329,
 'b_5577433': -342,
 'b_5577440': -90,
 'b_56323895': -87,
 'b_56325556': -94,
 'b_56325562': -85,
 'b_56325563': -178,
 'b_56325622': -87,
 'b_56325623': -81,
 'b_56325932': -453,
 'b_56325933': -592,
 'b_56326319': -323,
 'b_56326320': -4

In [52]:
shop_frame = pd.DataFrame(temp_shop_dict, index=['s_2871718'])
shop_frame = shop_frame/shop_frame.shape[1]
shop_frame
del temp_shop_dict

mall_frame = pd.DataFrame(temp_mall_dict, index=['m_1409'])
mall_frame = mall_frame/mall_frame.shape[1]
mall_frame
del temp_mall_dict

,b_1071564,b_12853924,b_13063943,b_15781664,b_19586106,b_25137939,b_25137940,b_25137941,b_25137942,b_26748506,...,b_57376407,b_5857369,b_5857370,b_6396479,b_6396480,b_6686445,b_8764723,b_8764724,b_8764725,b_8764726
s_2871718,-1.333333,-8.724638,-2.333333,-1.115942,-1.173913,-6.304348,-5.130435,-2.608696,-5.231884,-6.565217,...,-1.057971,-46.84058,-51.753623,-36.826087,-38.42029,-1.101449,-8.869565,-1.347826,-0.913043,-2.536232


,b_10004577,b_10004588,b_10063227,b_10066720,b_10071970,b_10083821,b_1009121,b_1009122,b_10093712,b_10097159,...,b_9761636,b_9809138,b_9837892,b_9893403,b_9926786,b_9935522,b_9964950,b_9972974,b_9983657,b_9986470
m_1409,-0.010293,-0.032431,-0.010434,-0.010998,-0.014382,-0.010857,-0.043852,-0.043852,-0.012126,-0.012408,...,-0.046531,-0.025099,-4.229695,-0.698957,-0.023125,-0.014664,-0.233785,-0.021433,-0.023125,-0.037225


In [64]:
temp = pd.concat([mall_frame, shop_frame]).fillna(0)
temp.iloc[1].sum()
shop_frame.iloc[0].sum()
temp

-619.8405797101449

-619.8405797101449

,b_10004577,b_10004588,b_10063227,b_10066720,b_10071970,b_10083821,b_1009121,b_1009122,b_10093712,b_10097159,...,b_9761636,b_9809138,b_9837892,b_9893403,b_9926786,b_9935522,b_9964950,b_9972974,b_9983657,b_9986470
m_1409,-0.010293,-0.032431,-0.010434,-0.010998,-0.014382,-0.010857,-0.043852,-0.043852,-0.012126,-0.012408,...,-0.046531,-0.025099,-4.229695,-0.698957,-0.023125,-0.014664,-0.233785,-0.021433,-0.023125,-0.037225
s_2871718,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [90]:
def spa_mall(x):
    mall_dict = mall_wifi_dict(x)
    mall_frame = pd.DataFrame(mall_dict, index=[x])
    mall_frame = mall_frame/mall_frame.shape[1]
    
    temp = full_data[full_data.mall_id == x].index
    temp = list(temp)
    for a in tqdm(temp):
        s_id = full_data.iloc[a].shop_id
        shop_dict = shop_wifi_dict(s_id)
        shop_frame = pd.DataFrame(shop_dict, index=[s_id])
        shop_frame = shop_frame/shop_frame.shape[1]
        if s_id not in mall_frame.index:
            mall_frame = pd.concat([mall_frame, shop_frame]).fillna(0)
        else:
            shop_frame = (shop_frame + mall_frame.loc[s_id])/2
            mall_frame = mall_frame.drop([s_id])
            mall_frame = pd.concat([mall_frame, shop_frame]).fillna(0)
    return mall_frame

In [93]:
m_2058_frame = spa_mall('m_2058')
m_2058_frame


  0%|          | 1/4392 [00:00<38:16,  1.91it/s]

KeyboardInterrupt: 

# Similarity

In [92]:
import math

In [21]:
def Cosine(x, y):
    x = np.array(x)
    y = np.array(y)
    return float(x.dot(y)/(math.sqrt((x**2).sum()) * math.sqrt((y**2).sum())))

In [37]:
def Multiply(x, y):
    sum_xy = 0.0
    for i in range(len(x)):
        temp = x[i]*y[i]
        sum_xy = sum_xy + temp
    return float(sum_xy)

def Pearson(x, y):
    n = len(x)
    sum_x = sum(x)
    sum_y = sum(y)
    sum_xy = Multiply(x, y)
    sum_x2 = sum([i**2 for i in x])
    sum_y2 = sum([j**2 for j in y])
    m = sum_xy - (float(sum_x) * float(sum_y)/n)
    d = math.sqrt((sum_x2-float(sum_x**2)/n) * (sum_y2-float(sum_y**2)/n))
    return float(m/d)

In [26]:
full_data.head(10)

,row_id,user_id,mall_id,wifi_infos,shop_id,wifi_num
0,-1,u_376,m_1409,"[[b_41124514, -86, false], [b_28723327, -90, f...",s_2871718,10
1,-1,u_376,m_1409,"[[b_6396480, -67, false], [b_56328155, -73, fa...",s_2871718,10
2,-1,u_1041,m_4079,"[[b_8006367, -78, false], [b_2485110, -52, fal...",s_181637,10
3,-1,u_1158,m_6587,"[[b_26250579, -73, false], [b_26250580, -64, f...",s_609470,10
4,-1,u_1654,m_3005,"[[b_39004150, -66, false], [b_39004148, -58, f...",s_3816766,10
5,-1,u_2733,m_3839,"[[b_13784698, -75, false], [b_7390858, -86, fa...",s_3686420,10
6,-1,u_2848,m_1920,"[[b_975810, -33, true], [b_975807, -41, false]...",s_89874,10
7,-1,u_3063,m_2878,"[[b_15263246, -67, false], [b_10120963, -76, f...",s_939447,10
8,-1,u_3063,m_2878,"[[b_37749132, -77, false], [b_25781349, -63, f...",s_939447,10
9,-1,u_3063,m_2878,"[[b_39846139, -73, false], [b_25673942, -61, f...",s_3698335,10


In [39]:
temp_shop_dict1 = shop_wifi_dict('s_939447')
temp_shop_dict2 = shop_wifi_dict('s_3698335')

shop_frame1 = pd.DataFrame(temp_shop_dict1, index=['s_939447'])
shop_frame2 = pd.DataFrame(temp_shop_dict2, index=['s_3698335'])

shop_frame = pd.concat([shop_frame1, shop_frame2])
shop_frame = shop_frame.fillna(0)
shop_frame
shop_frame = shop_frame.as_matrix()

,b_10120963,b_1017266,b_1017268,b_10174986,b_10704353,b_1077647,b_1077648,b_11247797,b_11247800,b_11856974,...,b_55555178,b_56174991,b_57110874,b_6617957,b_7420001,b_7421656,b_8189306,b_8931456,b_9610393,b_9691098
s_939447,-76.0,-280.0,-540.0,-70.0,0.0,0.0,0.0,0.0,0.0,-237.0,...,-140.0,0.0,0.0,-226.0,-59.0,-68.0,0.0,-60.0,-68.0,0.0
s_3698335,0.0,0.0,0.0,0.0,-51.0,-74.0,-84.0,-108.0,-106.0,0.0,...,0.0,-148.0,-120.0,0.0,0.0,0.0,-72.0,0.0,0.0,-146.0


In [40]:
Cosine(shop_frame[0], shop_frame[1])

Pearson(shop_frame[0], shop_frame[1])

0.32379822301709432

0.1598814924805109